In [1]:
import os
import math

import numpy as np
import pandas as pd
from scipy.io import loadmat
from scipy.signal import cheby2, sosfilt, resample
from tqdm.notebook import tqdm


dohko = pd.read_csv('../workdata/dohko/dohko.csv')
print(dohko.shape)
dohko.head()

/home/renan/Área de Trabalho/research-project/py37/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


(33169, 6)


,db,filename,diag,total_diags,db_freq,time
0,WFDB_CPSC2018,A4644.mat,SR,1,500,10.0
1,WFDB_PTBXL,HR15248.mat,SR,1,500,10.0
2,WFDB_Ga,E02092.mat,SR,1,500,10.0
3,WFDB_PTBXL,HR18712.mat,SR,5,500,10.0
4,WFDB_PTBXL,HR21119.mat,AF,4,500,10.0


In [2]:
n_rows = dohko.shape[1]
dohko['data'] = [[] for _ in range(n_rows)]

In [4]:
new_freq = 100

for index, row in tqdm(dohko.iterrows()):
    db = row['db']
    filename = row['filename']
    db_freq = row['db_freq']
    path = f'../databases/{db}/{filename}'
    record_file = loadmat(path)['val']
    recording = np.array(record_file, dtype=np.float64)
    n_leads, rec_samples = recording.shape 
    for lead in range(0, n_leads):
        sos = cheby2(12, 20, [0.35, 70], 'bandpass', fs=db_freq, output='sos')
        recording[lead, :] = sosfilt(sos, recording[lead, :])
    rec_time_sec = rec_samples/db_freq
    new_rec_sampling = int(rec_time_sec*new_freq)
    recording = resample(recording, new_rec_sampling, axis=1)
    dohko.at[index, 'data'] = recording

0it [00:00, ?it/s]

In [54]:
for index, row in enumerate(np.array_split(dohko, 10)):
    row.to_pickle(f'../workdata/miro/part_{index}.pkl')

### Flush RAM

In [ ]:
os._exit(0)